In [1]:
from flask import Flask, render_template, request,jsonify
# from flask_cors import CORS,cross_origin
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import Request,urlopen as uReq
import pymongo
dbConn = pymongo.MongoClient("mongodb://localhost:27017/")  # opening a connection to Mongo
db = dbConn['crawlerDB'] # connecting to the database called crawlerDB
table = db['TELEGRAPH']
app = Flask(__name__)  # initialising the flask app with the name 'app'
for x in range(1,100):
    tg_url="https://www.telegraphindia.com/search/coronavirus?&page="+str(x)
    #req=Request(tg_url, headers={'User-Agent': 'Mozilla/5.0'})
    uclient=uReq(tg_url)
    tgPage=uclient.read()
    uclient.close()
    tg_html=bs(tgPage,"html.parser")
    before=tg_html.find('ul',{'class':"listing-withImage"})
    bigboxes=before.find_all('li')
    try:
        for box in bigboxes:
                news_link= "https://www.telegraphindia.com"+box.h3.a['href']
                news_res=requests.get(news_link)
                news_html=bs(news_res.text,"html.parser")
                news_article=news_html.find('div',{'class':"padiingDetails story-advertise"})
                final_news=news_article.find_all('p')
                var=""
                for out in final_news:
                    var=var+out.text
                mydict={"news":var}
                table.insert_one(mydict) 
    except:
        continue

In [2]:
len(bigboxes)

6

In [4]:
print(news_link)

https://www.telegraphindia.com/world/coronavirus-united-states-begins-stockpiling-malaria-medicine-for-treating-covid-19-patients/cid/1760805


In [7]:
len(final_news)

19

The US has started procuring malaria medicine 'Hydroxychloroquine' for treating COVID-19 patients as it anticipates a favourable result from its ongoing treatment of some 1,100 coronavirus infected people in New York.
Pharmaceutical company Sandoz has donated 30 million doses of Hydroxychloroquine to the US government, president Donald Trump told reporters.
Bayer has donated one million doses of the chloroquine which will soon be distributed to states and health officials around the country, he said, adding that Teva Pharmaceuticals is also donating six million doses to US hospitals.
A day earlier, Trump said that malaria medicine Hydroxychloroquine is being administered to 1,100 coronavirus patients in New York, exuding confidence that the drug, touted as a "game-changer" by him, might give some incredible results in the fight against the disease that has infected over 163,000 people in the country.
It is being tested out on the COVID-19 patients in New York on an experimental basis, 